In [597]:
from scipy.optimize import *
import pandas as pd 
import numpy as np 
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from tqdm import tqdm
from pulp import *
import plotly.express as px 
from sklearn.decomposition import PCA
import random
import plotly.io as pio 
pio.renderers.default = "browser"
import plotly.graph_objects as go
from pandas_datareader import data

In [919]:
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/52))-1
        
        return(Result)

In [1086]:
df = pd.read_csv("ETFs_info.csv", index_col="Ticker")
df["Inception"] = pd.to_datetime(df["Inception"])
df = df[df["Inception"] <= "2015-01-01"]

fundamental_df = pd.read_csv("fund_risk_cluster_reduced.csv",index_col="Ticker") #Fundamental Clustering Data
weekly_return = pd.read_csv("WeeklyReturns.csv",index_col="Date") #Weekly Return Data
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(weekly_return.columns)]
fundamental_df = fundamental_df.loc[fundamental_df.index.intersection(df.index)]
len(fundamental_df)

695

,Category,Fundamental Cluster,Risk Cluster
SPY,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
IVV,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VTI,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VOO,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
QQQ,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
...,...,...,...
NFTY,India Equity,Low Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility
FTAG,Natural Resources,Mid Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility
CHII,China Region,Low Fundamentals Broad Market,Moderat Risk - Slighty Below Market Volatility
CHIM,China Region,Low Fundamentals Broad Market,Moderat/High Risk - Slightly Above Market Vola...


In [1087]:
ticks = list(fundamental_df.index)

In [1088]:

# startDate = "2015-01-01"
# endDate = "2021-01-01"
# # DOWNLOAD THE DATA FROM YAHOO DATABASE
# #------------------------------------------------------------------
# dailyPrices = data.DataReader(ticks, 'yahoo', startDate, endDate)
# dailyPrices = dailyPrices["Adj Close"]

In [1089]:
# dailyPrices.to_csv("Dailyprices.csv", index="Date")
# daily_prices = pd.read_csv("DailyPrices.csv", index_col="Date")

In [937]:
daily_prices.dropna(axis=1, inplace=True)
daily_prices

In [944]:
weekly_returns = daily_prices.pct_change().dropna(axis=0)

In [951]:
def calculate_pdi( weekly_returns, portfolios): 
    
    def meanRetAn(data):             
        Result = 1
        
        for i in data:
            Result *= (1+i)
            
        Result = Result**(1/float(len(data)/30))-1
        
        return(Result)

    pca = PCA()
    PDI_dict = {}

    for i,y in tqdm(zip(portfolios,range(1,len(portfolios)+1))):
        n_assets = len(i)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_weekly_return = weekly_returns[i].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        ann_ret = meanRetAn(list(port_weekly_return))
        an_cov = weekly_returns[i].cov()
        port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(30)
        corr_matrix = np.array(weekly_returns[i].corr().replace(np.nan, 0))
        principalComponents = pca.fit(corr_matrix)
        PDI = 2*sum(principalComponents.explained_variance_ratio_*range(1,len(principalComponents.explained_variance_ratio_)+1,1))-1
        PDI_dict[y] = {}
        PDI_dict[y]["PDI_INDEX"] = PDI
        PDI_dict[y]["# of Assets"] = len(i)
        PDI_dict[y]["Assets"] = i
        PDI_dict[y]["Sharpe Ratio"] = ann_ret/port_std
        PDI_dict[y]["Annual Return"] = ann_ret
        PDI_dict[y]["Annual STD"] = port_std
    

        


    PDI_DF = pd.DataFrame(PDI_dict).T
    #PDI_DF["Assets"] = PDI_DF["Assets"].astype(str)
    PDI_DF["# of Assets"] = PDI_DF["# of Assets"].astype(str)
    PDI_DF["Sharpe Ratio"] = PDI_DF["Sharpe Ratio"].astype(float)
    PDI_DF["Annual STD"] = PDI_DF["Annual STD"].astype(float)
    PDI_DF["PDI_INDEX"] = PDI_DF["PDI_INDEX"].astype(float)
    PDI_DF["Annual Return"] = PDI_DF["Annual Return"].astype(float)
    SPY_DF = PDI_DF.iloc[0,:]
    return PDI_DF,SPY_DF

In [1083]:
def pca_per(return_data, initial_port, interval):

    data = return_data # data containing weekly returns

    data.index = pd.to_datetime(data.index) # Conveting the index which is date to datetime
    weeks_list = data.index # grabbing all index dates
    data.index = data.index.to_period(interval) # converting the index to quarterly sets
    periods = data.index.unique() # taking the unique quarters to loop over
    
    print(periods)
    first_period = periods[0] # the first period of the time frame
    remaining_periods = periods[1:] # the remianing periods of the time framr
    first_periods = periods[:-1] # all periods minus the last

    performance = []
    sharpe_performance = []



    def port_ret(returns,port, period): # function for calculating returns
        n_assets = len(initial_port)
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_return = returns.loc[period][port].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        return  port_return

    ############################################################ Calculate first period ####################################################################################
    performance.extend(port_ret(data, initial_port,first_period))
    sharpe_performance.extend(port_ret(data, initial_port,first_period))
    tickers_weekly = list(data.columns)
    number_of_assets = [len(initial_port)]



    ################################################################## Portfolio Creation ###################################################################################
    samples = [["SPY"]]
    for number in [len(initial_port)]:
        for i in range(1,30000):
            #samples.extend([list(x) for x in combinations(selected_tickers, number_of_assets)])
            samples.append(random.sample(list(tickers_weekly),number))
    samples_mini = []
    for i in samples:
        if i not in samples_mini:
            samples_mini.append(i)

    print("Number of Portfolios: {}".format(len(samples_mini)))

    vol_d = {}
    ######################################################## Calculation of portfolio perfomnce #############################################################################

    for init_time, next_time in zip(first_periods,remaining_periods):
        print("first time: {} - last time: {}".format(init_time, next_time))
        PDI_DF, SPY_DF = calculate_pdi(weekly_returns = data.loc[init_time].dropna(axis=1), portfolios = samples_mini)

        q1,q2 = PDI_DF["PDI_INDEX"].quantile([0.20,0.85])
        r1,r2 = PDI_DF["Annual Return"].quantile([0.4,0.85])


        #Taking the higest return within the range
        SPY_vol_lower = SPY_DF["Annual STD"] - 0.04 # allowing for a little room of divergence to find a asset match
        SPY_vol_upper = SPY_DF["Annual STD"] + 0.01# allowing for a little room of divergence to find a asset match

        PDI_rev = PDI_DF[(PDI_DF["Annual STD"] >= SPY_vol_lower) & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["PDI_INDEX"] >= q1)& (PDI_DF["PDI_INDEX"] <= q2)\
                         & (PDI_DF["Annual STD"] <= SPY_vol_upper) & (PDI_DF["Annual Return"] >= r1) & (PDI_DF["Annual Return"] <= r2)].reset_index().drop(columns="index")
                        # & (PDI_DF["PDI_INDEX"] >= q1) ].reset_index().drop(columns="index")


        # fig = px.scatter(PDI_DF,  color="Annual STD", y="Annual Return", x="PDI_INDEX")
        # fig.show()


        ################################################## Taking the higest return within the range ##########################################################################

        id = PDI_rev["Annual Return"].idxmax()
        port_max_ret = PDI_rev["Assets"][id]
        
        port_max_ret_period = port_ret(data,port_max_ret, next_time)
        performance.extend(port_max_ret_period)

        ################################################### Taking the best Sharpe Ratio within the range ######################################################################

        id_sharpe = PDI_rev["Sharpe Ratio"].idxmax()
        port_max_sharpe = PDI_rev["Assets"][id_sharpe]
        
        port_max_sharpe_period = port_ret(data,port_max_sharpe, next_time)
        sharpe_performance.extend(port_max_sharpe_period)

    dd = pd.DataFrame()
    dd["Time"] = time
    dd["Per"] = per
    dd["SPY"] = list(weekly_return["SPY"])
    dd["Sharpe"] = sharpe
    dd["Per_cum"] = dd["Per"].cumsum(axis=0)
    dd["SPY_cum"] = dd["SPY"].cumsum(axis=0)
    dd["Sharpe_cum"] = dd["Sharpe"].cumsum(axis=0)
    fig = px.line(dd, x="Time", y = ["Per_cum","SPY_cum","Sharpe_cum"], title="Number of Assets: {} - period interval {}".format(len(initial_port), interval))




    return weeks_list, performance, sharpe_performance , fig, dd




In [1084]:
ti = fundamental_df[(-fundamental_df["Category"].isin(['Trading--Leveraged Equity','Trading--Inverse Equity']) & (fundamental_df["Risk Cluster"].isin(['Moderat/High Risk - Slightly Above Market Volatility'])))]
tttt = list(ti.index)
weekly_return = daily_prices.pct_change().dropna(axis=0)

In [1085]:
time, per, sharpe, fig , performance_frame= pca_per(weekly_return[tttt],['SPY','IVV','VTI','VOO'], interval="Q")
fig.show()

PeriodIndex(['2015Q1', '2015Q2', '2015Q3', '2015Q4', '2016Q1', '2016Q2',
             '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4',
             '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2',
             '2019Q3', '2019Q4'],
            dtype='period[Q-DEC]', name='Date', freq='Q-DEC')
36it [00:00, 350.76it/s]Number of Portfolios: 30000
first time: 2015Q1 - last time: 2015Q2
30000it [01:23, 360.16it/s]
71it [00:00, 352.37it/s]first time: 2015Q2 - last time: 2015Q3
30000it [01:22, 362.63it/s]
71it [00:00, 351.21it/s]first time: 2015Q3 - last time: 2015Q4
30000it [01:23, 358.82it/s]
36it [00:00, 354.67it/s]first time: 2015Q4 - last time: 2016Q1
30000it [01:23, 361.36it/s]
70it [00:00, 345.74it/s]first time: 2016Q1 - last time: 2016Q2
30000it [01:22, 363.24it/s]
72it [00:00, 355.63it/s]first time: 2016Q2 - last time: 2016Q3
30000it [01:22, 364.32it/s]
35it [00:00, 346.12it/s]first time: 2016Q3 - last time: 2016Q4
30000it [01:22, 364.20it/s]
71it [00:00, 353.46i

In [495]:
#dddd = PDI_DF[PDI_DF["# of Assets"] == "14" ]
dddd = PDI_DF.copy()
#dddd = PDI_DF[PDI_DF["Annual STD"] <= 0.2 ]

In [496]:
dddd["PDI SEG"] = dddd["PDI_INDEX"].apply(lambda x: str(np.round(x, 1)))
dddd["STD SEGG"] = dddd["Annual STD"].apply(lambda x: np.round(x, 2))

In [497]:
dddd

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,PDI SEG,STD SEGG
1,NaN,1,[SPY],0.434199,0.041268,0.095045,nan,0.10
2,1.143835,3,"[GLIN, FDIS, URTH]",0.492642,0.063373,0.128638,1.1,0.13
3,1.403781,3,"[VDE, SPXL, EFG]",0.252745,0.045734,0.180947,1.4,0.18
4,1.193516,3,"[ENFR, FEX, IGN]",0.020559,0.002275,0.110633,1.2,0.11
5,1.206487,3,"[BICK, EWO, VEA]",0.260815,0.039640,0.151986,1.2,0.15
...,...,...,...,...,...,...,...,...
119991,1.985287,8,"[IAK, KCE, EEMA, FAS, ROBO, FPX, USD, IHDG]",0.244595,0.034916,0.142752,2.0,0.14
119992,2.584147,8,"[FLN, KFYP, HEFA, PSCC, PID, XSVM, QDEF, CXSE]",-0.181389,-0.023573,0.129960,2.6,0.13
119993,2.504402,8,"[VTWV, EXI, FDT, FXU, IHDG, KBWR, VONE, SCHC]",0.788838,0.077643,0.098427,2.5,0.10
119994,2.303937,8,"[IGE, VOOV, SDOG, ARKW, IEV, BFOR, VTWO, FCG]",0.186711,0.024852,0.133101,2.3,0.13


In [459]:
print("Min STD:", dddd["Annual STD"].min())
print("Max STD:", dddd["Annual STD"].max())

Min STD: 0.034345568498662105
Max STD: 0.4010084942247244


In [482]:
fig = px.bar(dddd, color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [483]:
fig = px.scatter(dddd,  color="# of Assets", y="Annual Return", x="PDI_INDEX")
fig.show()

In [486]:
fig = px.box(dddd, x="# of Assets", y="PDI_INDEX")
fig.show()

In [498]:
fig = px.scatter_3d(dddd, x='STD SEGG', y = "Annual Return", z = "PDI_INDEX", color="# of Assets", hover_data = [dddd.index], title = "ETF Portfolios")
fig.show()

In [462]:
fig = px.scatter(dddd, x="PDI_INDEX", y="Annual Return", color = "STD SEG")
fig.show()

In [366]:
testest = dddd[dddd["# of Assets"] == "5"]
print(len(testest))
testest.head()

1999


,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
5994,1.044022,5,"[SQQQ, EMLP, IGM, EQL, IWB]",-0.584013,-0.042722,0.073153
5995,1.381763,5,"[QDF, FPX, IUSG, ICOL, FLN]",0.452782,0.094287,0.208239
5996,1.747769,5,"[FEUZ, QDF, VIOG, IGE, RWL]",0.370002,0.073438,0.198480
5997,2.163634,5,"[EDIV, SCHA, FCG, NXTG, EQL]",0.110250,0.024112,0.218707
5998,1.895498,5,"[EWMC, QTEC, EFA, FRAK, CHIE]",0.219108,0.048795,0.222700


In [441]:
testest["Annual STD"]

5994    0.073153
5995    0.208239
5996    0.198480
5997    0.218707
5998    0.222700
          ...   
7988    0.217509
7989    0.163114
7990    0.202811
7991    0.183826
7992    0.156315
Name: Annual STD, Length: 1999, dtype: float64

In [368]:
fig = px.scatter_3d(testest, x='Annual STD', y = "Annual Return", z = "PDI_INDEX", color="Sharpe Ratio", hover_data = [testest.index], title = "ETF Portfolios")
fig.show()

In [382]:
indx = [7133,7136, 7494, 6177, 7703]

In [383]:
test = dddd.iloc[indx,:]

In [500]:
52/4

13.0

In [385]:
for i in test["Assets"]:
    print(fundamental_df.loc[i,:]["Category"])
    print(30*"-")

PSJ       Technology
ARKW      Technology
VTV      Large Value
EQWL     Large Blend
SPYG    Large Growth
Name: Category, dtype: object
------------------------------
PIZ     Foreign Large Growth
SMH               Technology
IXJ                   Health
ARKW              Technology
VPU                Utilities
Name: Category, dtype: object
------------------------------
ARKW              Technology
FSZ     Miscellaneous Region
NOBL             Large Blend
IWL              Large Blend
FPXI    Foreign Large Growth
Name: Category, dtype: object
------------------------------
FYC             Small Growth
IYH                   Health
IYW               Technology
ARKW              Technology
FPXI    Foreign Large Growth
Name: Category, dtype: object
------------------------------
SMH         Technology
XLU          Utilities
FTHI    Option Writing
IHI             Health
FVD        Large Value
Name: Category, dtype: object
------------------------------


In [272]:
num_as = []
max_pdi = []
for i in np.sort(dddd["# of Assets"].unique()):
    max_pdi.append(dddd[dddd["# of Assets"] == i]["PDI_INDEX"].max())
    num_as.append(i)
fig = px.line( x=num_as,y = max_pdi)
fig.show()

In [247]:

fig = px.scatter(dddd[dddd["# of Assets"] <= 3], y='Annual Return',  color ="PDI_RATIO",x = "Annual STD")
fig.show()

In [237]:
dddd

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,pdi_st,num_st,PDI_RATIO
1,NaN,1,[SPY],0.818855,0.130413,0.159263,0.100213,0.111111,0.901920
2,1.414592,3,"[SCJ, PSCM, SPVM]",0.461832,0.087222,0.188861,0.073067,0.111111,0.657605
3,1.302344,3,"[SOXL, PUI, XES]",0.416604,0.172723,0.414597,0.013145,0.111111,0.118301
4,1.054566,3,"[DGRS, RYJ, PBE]",0.355442,0.080435,0.226295,0.061685,0.111111,0.555161
5,1.255277,3,"[VSS, EUFN, EUDG]",0.287435,0.059219,0.206026,0.171623,0.111111,1.544605
...,...,...,...,...,...,...,...,...,...
33980,2.755384,19,"[QABA, SCHF, RZV, VT, VFH, UGE, FYX, XLP, EIS,...",0.665814,0.142286,0.213703,0.448405,1.000000,0.448405
33981,2.854350,19,"[JPIN, TUR, TILT, SYE, XLI, RDIV, IXUS, VTWO, ...",0.405457,0.077874,0.192065,0.641268,1.000000,0.641268
33982,3.651829,19,"[PWB, EWSC, VOOV, FLM, PBJ, EEM, EFA, EVX, ENZ...",0.465379,0.081121,0.174311,0.426395,1.000000,0.426395
33983,2.763340,19,"[VTWG, MIDU, RYE, EMLP, IYW, IYT, PSJ, XOP, GV...",0.377863,0.082463,0.218236,0.665928,1.000000,0.665928


In [8]:
fig = px.box(PDI_DF, x="# of Assets", y="Annual STD")
fig.show()

In [70]:
fig = px.box(PDI_DF, x="# of Assets", y="Annual Return")
fig.show()

In [9]:
print("min PDI",PDI_DF["PDI_INDEX"].min())
print("max PDI",PDI_DF["PDI_INDEX"].max())

min PDI 1.000214371065589
max PDI 5.135170565856279


In [10]:
PDI_DF

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
1,NaN,1,[SPY],0.818855,0.130413,0.159263
2,1.414592,3,"[SCJ, PSCM, SPVM]",0.461832,0.087222,0.188861
3,1.302344,3,"[SOXL, PUI, XES]",0.416604,0.172723,0.414597
4,1.054566,3,"[DGRS, RYJ, PBE]",0.355442,0.080435,0.226295
5,1.255277,3,"[VSS, EUFN, EUDG]",0.287435,0.059219,0.206026
...,...,...,...,...,...,...
33980,2.755384,19,"[QABA, SCHF, RZV, VT, VFH, UGE, FYX, XLP, EIS,...",0.665814,0.142286,0.213703
33981,2.854350,19,"[JPIN, TUR, TILT, SYE, XLI, RDIV, IXUS, VTWO, ...",0.405457,0.077874,0.192065
33982,3.651829,19,"[PWB, EWSC, VOOV, FLM, PBJ, EEM, EFA, EVX, ENZ...",0.465379,0.081121,0.174311
33983,2.763340,19,"[VTWG, MIDU, RYE, EMLP, IYW, IYT, PSJ, XOP, GV...",0.377863,0.082463,0.218236


In [11]:
a = PDI_DF["PDI_INDEX"].quantile(.5)
a

2.3683454956402015

In [42]:
 df_1 = PDI_DF[PDI_DF["PDI_INDEX"] <= a]
 df_1

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
2,1.210099,3,"[KRE, CHIQ, FPA]",0.564758,0.120226,0.212881
3,1.204287,3,"[FYT, SYG, MDYG]",0.563357,0.115168,0.204432
4,1.750332,3,"[MMTM, IAK, ZDEU]",0.523091,0.094329,0.180331
5,1.385366,3,"[FENY, DFJ, PYZ]",0.134866,0.030540,0.226449
6,1.134295,3,"[TOLZ, RFG, XLY]",0.537009,0.098377,0.183195
...,...,...,...,...,...,...
17969,1.313271,11,"[SPLV, EWC, QLD, DTD, XLV, EMQQ, EWM, PBE, SAA...",0.572785,0.098076,0.171227
17978,1.119875,11,"[SCHE, EUSA, JKI, DLS, ASEA, XMLV, DOO, SDY, M...",-0.147118,-0.020176,0.137139
17984,1.718354,11,"[SIL, XLG, QQQE, PSJ, FDT, RDVY, CN, SDIV, FNC...",0.684111,0.132154,0.193176
17987,1.192784,11,"[TOK, SMEZ, EZA, BBH, GXF, TNA, UDOW, EWT, PSQ...",0.425618,0.091966,0.216077


In [32]:
PDI_DF["Sharpe Ratio"].quantile([0.5])

0.5    0.497023
Name: Sharpe Ratio, dtype: float64

In [ ]:
# #0 - 0.25 quantile
# df_1 = PDI_DF[(PDI_DF["PDI_INDEX"] <= 2.04)]
# df_2 = PDI_DF[(PDI_DF["PDI_INDEX"] > 2.04) ]
# # #0.25 - 0.5 quantile
# # df_2 = PDI_DF[(PDI_DF["PDI_INDEX"] <= 1.647235)&(PDI_DF["PDI_INDEX"] > 1.181088) & (PDI_DF["Sharpe Ratio"] >= 0)]
# # #0-.5 - 0.75 quantile
# # df_3 = PDI_DF[(PDI_DF["PDI_INDEX"] > 1.647235)&(PDI_DF["PDI_INDEX"] <= 2.143900)& (PDI_DF["Sharpe Ratio"] >= 0)]
# # #0.75-1
# # df_4 = PDI_DF[(PDI_DF["PDI_INDEX"] > 2.143900) & (PDI_DF["Sharpe Ratio"] >= 0)]

In [24]:
qq_dict = {}
for i in PDI_DF["# of Assets"].unique():
    qq_dict[i] = {}
    dff_pdi = PDI_DF[PDI_DF["# of Assets"] == str(i)]
    q_mid = dff_pdi["PDI_INDEX"].quantile(0.5)
    df_1 = dff_pdi[dff_pdi["PDI_INDEX"] < q_mid]
    df_2 = dff_pdi[dff_pdi["PDI_INDEX"] >= q_mid]
    for frame,q in zip([df_1,df_2],["qq","qqq"]):
        qq_dict[i][q] = {}
        for ii in enumerate(frame["Assets"]):
            n_assets = len(ii[1])
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_return[ii[1]].mul(portfolio_weights_ew,axis=1).sum(axis=1)
            qq_dict[i][str(q)][str(ii[0])] = port_weekly_return




In [78]:
kk = pd.DataFrame(qq_dict["19"]["qqq"])
rr = pd.DataFrame(qq_dict["19"]["qq"])
hh = pd.merge(kk,rr, left_index=True, right_index=True, how = "inner", suffixes=("q2", "q1"))
hh["SPY"] = weekly_return["SPY"]
hh = hh.cumsum(axis=0)

In [79]:
hh

,0q2,1q2,2q2,3q2,4q2,5q2,6q2,7q2,8q2,9q2,...,990q1,991q1,992q1,993q1,994q1,995q1,996q1,997q1,998q1,SPY
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-14,0.007775,-0.006443,-0.004665,0.000477,0.001904,-0.002351,-0.002467,0.002573,-0.001553,-0.003189,...,0.003182,-0.007167,-0.004458,-0.000438,-0.001459,-0.003630,0.000753,-0.003448,-0.007850,-0.007167
2015-01-21,0.019609,0.018157,0.009731,0.012781,0.018746,0.011477,0.013109,0.014438,0.006667,0.020057,...,0.024829,0.006671,0.008422,0.014725,0.011357,0.015508,0.017013,0.002423,0.009033,0.003885
2015-01-28,0.019422,0.013044,0.000676,0.009937,0.009879,0.008846,0.009099,0.010368,0.006307,0.010229,...,0.012087,-0.006437,0.007548,0.001856,0.007293,0.010139,0.006578,-0.005796,0.002703,-0.010592
2015-02-04,0.029840,0.022598,0.021104,0.032842,0.017673,0.025161,0.027313,0.027370,0.025071,0.034355,...,0.024448,0.010395,0.023059,0.018182,0.023556,0.030310,0.023741,0.019626,0.029887,0.008994
2015-02-11,0.032241,0.021728,0.029591,0.043878,0.014953,0.035822,0.026608,0.036631,0.028914,0.039104,...,0.021024,0.020915,0.026820,0.025506,0.027275,0.030324,0.027915,0.032685,0.031113,0.023059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-02,0.700761,0.554838,0.614004,0.661659,0.401365,0.614587,0.557392,0.659134,0.616216,0.620634,...,0.820464,0.679828,0.602088,0.643798,0.526504,0.460735,0.617465,0.865595,0.567715,0.784790
2020-12-09,0.712979,0.571318,0.630498,0.687227,0.411983,0.632405,0.575009,0.674100,0.630944,0.637688,...,0.828879,0.693728,0.611798,0.653937,0.542460,0.469563,0.634005,0.889937,0.591209,0.784953
2020-12-16,0.726915,0.590932,0.640348,0.697796,0.422059,0.645983,0.590214,0.690358,0.643238,0.653759,...,0.842290,0.706456,0.621229,0.662681,0.550370,0.478575,0.644921,0.903803,0.603545,0.794003


In [80]:
fig = px.line(hh, x= hh.index, y=hh.columns)

In [81]:
fig.show()

In [184]:
pdd = PDI_DF.copy()
pdd = pdd[pdd["# of Assets"].isin(["5","10","17"])]

q_mid_5 = pdd[pdd["# of Assets"] == "5"].quantile(0.8)
q_mid_10 = pdd[pdd["# of Assets"] == "10"].quantile(0.8)
q_mid_17 = pdd[pdd["# of Assets"] == "17"].quantile(0.8)


pdd

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD
4000,1.742603,5,"[FYX, PRFZ, PIZ, TDIV, FXU]",0.525615,0.095345,0.181397
4001,1.728121,5,"[FENY, VUG, ASHR, EWP, VTI]",0.333801,0.065499,0.196222
4002,1.457414,5,"[EQAL, JKJ, VXUS, FXZ, HEZU]",0.442982,0.088362,0.199472
4003,1.670686,5,"[SPXL, FDTS, ROBO, EWT, SIL]",0.664589,0.166872,0.251090
4004,1.505472,5,"[DFE, SPYG, QLD, FTAG, MDYG]",0.622211,0.134028,0.215407
...,...,...,...,...,...,...
29982,2.989923,17,"[EMLP, VDC, PSCU, RCD, EWUS, UDOW, SPUU, MOO, ...",0.555782,0.112383,0.202207
29983,1.084684,17,"[RYT, CHIX, SCZ, XAR, IVOG, HDV, FNDB, SDOW, R...",0.341905,0.050703,0.148296
29984,1.410259,17,"[RTH, IGF, EWU, QYLD, FAD, IHF, DVYA, IDHQ, DO...",0.295736,0.040002,0.135263
29985,3.202143,17,"[XRT, SCZ, BFOR, CIZ, IJJ, SLY, PIE, BRF, RYU,...",0.411725,0.079728,0.193643


In [183]:
def upper_lower(df):
    if df["# of Assets"] == "5":
        if df["PDI_INDEX"] < q_mid_5:
            return "Lower_5"
        else:
            return "Upper_5"
    elif df["# of Assets"] == "10":
        if float(df["PDI_INDEX"]) < q_mid_10:
            return  "Lower_10"
        else:
            return "Upper_10"
    elif df["# of Assets"] == "17":
        if df["PDI_INDEX"] < q_mid_17:
            return "Lower_17"
        else:
            return  "Upper_17" 

In [182]:
pdd["QQ"] = pdd.apply(upper_lower)

KeyError: '# of Assets'

In [145]:
low_q = df_1
low_q["q"] = "Low_div"
low_q["q"] = low_q["q"]+"-"+low_q["# of Assets"]
low_q

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,q
4000,1.742603,5,"[FYX, PRFZ, PIZ, TDIV, FXU]",0.525615,0.095345,0.181397,Low_div-5
4001,1.728121,5,"[FENY, VUG, ASHR, EWP, VTI]",0.333801,0.065499,0.196222,Low_div-5
4002,1.457414,5,"[EQAL, JKJ, VXUS, FXZ, HEZU]",0.442982,0.088362,0.199472,Low_div-5
4003,1.670686,5,"[SPXL, FDTS, ROBO, EWT, SIL]",0.664589,0.166872,0.251090,Low_div-5
4004,1.505472,5,"[DFE, SPYG, QLD, FTAG, MDYG]",0.622211,0.134028,0.215407,Low_div-5
...,...,...,...,...,...,...,...
29981,1.727494,17,"[ARKK, FUTY, FILL, WBIG, ENZL, DXJS, REMX, EFA...",0.375174,0.049709,0.132495,Low_div-17
29982,2.989923,17,"[EMLP, VDC, PSCU, RCD, EWUS, UDOW, SPUU, MOO, ...",0.555782,0.112383,0.202207,Low_div-17
29983,1.084684,17,"[RYT, CHIX, SCZ, XAR, IVOG, HDV, FNDB, SDOW, R...",0.341905,0.050703,0.148296,Low_div-17
29984,1.410259,17,"[RTH, IGF, EWU, QYLD, FAD, IHF, DVYA, IDHQ, DO...",0.295736,0.040002,0.135263,Low_div-17


In [138]:
# for i in low_q["Assets"][1:]:
#     print(fundamental_df.loc[i,:]["Category"].value_counts())

In [146]:
high_q = df_2
high_q["q"] = "High_div"
high_q["q"] = high_q["q"]+"-"+high_q["# of Assets"]
high_q

,PDI_INDEX,# of Assets,Assets,Sharpe Ratio,Annual Return,Annual STD,q
4966,3.117902,5,"[ECH, EPU, GREK, ZJPN, EVX]",0.249006,0.050331,0.202129,High_div-5
5300,3.110198,5,"[SOXL, EPOL, IXP, FDL, EWUS]",0.673711,0.190236,0.282370,High_div-5
13995,3.684217,10,"[EDEN, EWI, TUR, LTL, DTH, EMQQ, VV, XES, CURE...",0.308041,0.072221,0.234453,High_div-10
13996,3.570942,10,"[BKF, VEGI, SDIV, FLN, ROBO, IYE, PXF, DBEZ, B...",0.307772,0.065578,0.213074,High_div-10
14000,3.962951,10,"[QAT, PID, WBIL, PGAL, SPDW, KWEB, XPH, IDU, V...",0.418949,0.064205,0.153253,High_div-10
...,...,...,...,...,...,...,...
29969,3.124974,17,"[KRE, GMF, CHIC, RXI, ADRE, IWO, IWF, DGRW, XL...",0.554759,0.101371,0.182730,High_div-17
29971,3.080922,17,"[URTH, EPP, DBEZ, SOXX, QQEW, FILL, EQL, FDIS,...",0.550808,0.104083,0.188964,High_div-17
29973,3.268873,17,"[EWD, RPG, IGV, CHII, WBIG, FNI, PSCD, KWEB, M...",0.728478,0.126320,0.173402,High_div-17
29977,4.055728,17,"[SKYY, FGM, SCHC, BICK, IJK, FYX, OIH, EFG, IW...",0.419319,0.084942,0.202571,High_div-17


In [131]:
# for i in high_q["Assets"][1:]:
#     print(fundamental_df.loc[i,:]["Category"].value_counts())

In [148]:
concat_frame = pd.concat([high_q,low_q])

In [149]:
fig = px.box(concat_frame, x="q", y="Annual STD")
fig.show()

In [150]:
fig = px.box(concat_frame, x="q", y="Annual Return")
fig.show()

In [151]:
fig = px.box(concat_frame, x="q", y="Sharpe Ratio")
fig.show()

In [101]:
fundamental_df.head()

,Category,Fundamental Cluster,Risk Cluster
SPY,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
IVV,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VTI,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
VOO,Large Blend,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...
QQQ,Large Growth,High Fundamental Broad Market,Moderat/High Risk - Slightly Above Market Vola...


In [62]:
testtest = pd.DataFrame()

for i in PDI_DF["# of Assets"].unique():
    testtest["qq-{}".format(i)] = pd.DataFrame(qq_dict[i]["qq"]).mean(axis=1)
    testtest["qqq-{}".format(i)] = pd.DataFrame(qq_dict[i]["qqq"]).mean(axis=1)
    

In [63]:
testtest.dropna(axis=1, inplace=True)
testtest["SPY"] = weekly_return["SPY"]
testtest = testtest.cumsum(axis=0)
fig = px.line(testtest, x= testtest.index, y=testtest.columns)
fig.show()

In [34]:
testtest = pd.DataFrame()
fig = px.line()




for i in PDI_DF["# of Assets"].unique():
    
    testtest = pd.DataFrame()
    testtest["q1"] = pd.DataFrame(qq_dict[i]["qq"]).mean(axis=1)
    testtest["q2"] = pd.DataFrame(qq_dict[i]["qqq"]).mean(axis=1)
    testtest = testtest.cumsum(axis=0)
    fig.add_trace(go.Scatter(x=testtest.index, y=testtest.columns, mode="lines"))
fig.show()

In [26]:
qq_dict[3]

KeyError: 3

In [ ]:
    testtest = pd.DataFrame()
    testtest["q1"] = pd.DataFrame(qq_dict["q1"]).mean(axis=1)
    testtest["q2"] = pd.DataFrame(qq_dict["q2"]).mean(axis=1)
    testtest = testtest.cumsum(axis=0)
    testtest["id"] = testtest.index
    fff = pd.wide_to_long(testtest,stubnames="q", i="id", j="quantile")
    fff = fff.reset_index().set_index("id")
    print(fff)
    fig = px.line(fff, x= fff.index, y="q")
    fig.show()


In [ ]:
len(df_1)

In [ ]:
weekly_return.index = pd.to_datetime(weekly_return.index)

In [ ]:
qq = []
year = []
porti = []
num_as = []
sr = []
ar = []
std = []
for i in list(weekly_return.index.year.unique()):
    for dfs,q in zip([df_1,df_2], ["q1","q2"]):
        for port in dfs["Assets"]:
            n_assets = len(port)
            portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
            port_weekly_return = weekly_return[weekly_return.index.year == i][port].mul(portfolio_weights_ew,axis=1).mean(axis=1)
            ann_ret = meanRetAn(list(port_weekly_return))
            an_cov = weekly_return[weekly_return.index.year == i][port].cov()
            port_std = np.sqrt(np.dot(portfolio_weights_ew.T, np.dot(an_cov, portfolio_weights_ew)))*np.sqrt(52)
            qq.append(q)
            year.append(i)
            porti.append(port)
            num_as.append(len(port))
            sr.append(ann_ret/port_std)
            ar.append(ann_ret)
            std.append(port_std)


In [ ]:
df = pd.DataFrame()
df["year"] = year
df["q"] = qq
df["aret"] = ar
df["prt"] = porti
df["std"] = std
df["num_p"] = num_as
df["prt"] = df["prt"].astype(str)
df["aret"] = df["aret"].astype(float)

In [ ]:
dff = df.groupby(["year", "q"])[["aret","std"]].mean()
dff = pd.DataFrame(dff).reset_index()

In [ ]:
fig = px.bar(dff, x='year', y='aret', color="q",barmode='group', hover_data = ["std"])
fig.show()

In [ ]:
df_1.head()

In [ ]:
px.histogram(df_2, x="Annual Return")

In [173]:

qq_dict = {}

for i,q in zip([df_1,df_2], ["q1","q2"]):
    qq_dict[q] = {}
    for ii in enumerate(i["Assets"]):
        n_assets = len(ii[1])
        portfolio_weights_ew = np.repeat(1/n_assets, n_assets)
        port_weekly_return = weekly_return[ii[1]].mul(portfolio_weights_ew,axis=1).sum(axis=1)
        qq_dict[q][ii[0]] = port_weekly_return



In [174]:
testtest = pd.DataFrame()
testtest["q1"] = pd.DataFrame(qq_dict["q1"]).mean(axis=1)
testtest["q2"] = pd.DataFrame(qq_dict["q2"]).mean(axis=1)
testtest = testtest.cumsum(axis=0)


In [175]:
testtest

,q1,q2
Date,,
2015-01-14,-0.002192,-0.001945
2015-01-21,0.013071,0.014089
2015-01-28,0.007894,0.008584
2015-02-04,0.024714,0.026921
2015-02-11,0.027821,0.031518
...,...,...
2020-12-02,0.570970,0.631153
2020-12-09,0.585042,0.647037
2020-12-16,0.597654,0.661405


In [176]:
fig = px.line(testtest, x= testtest.index, y=testtest.columns )
fig.show()

In [ ]:
tickers = list(fundamental_df.index)
startDate = "2015-01-01"
endDate = "2021-01-01"
dailyPrices = data.DataReader(tickers, 'yahoo', startDate, endDate)

In [ ]:
dailyPrices = dailyPrices["Adj Close"]

In [ ]:
dailyPrices.to_csv("DailyPrices.csv")

In [ ]:
pricesWed = dailyPrices[dailyPrices.index.weekday==2] 

 weeklyPrices = pricesWed.dropna(axis=1)                 #delete NaN columns
    
# GET WEEKLY RETURNS
weeklyReturns = weeklyPrices.pct_change()                  
weeklyReturns = weeklyReturns.drop(weeklyReturns.index[:1]) 

weeklyReturns.to_csv("WeeklyReturns.csv")